# Classes Round Three

Each data point is a class instance.

- add get period, get only period to base class
- do cum sum stuff

In [1]:
# Import statements
import pandas as pd
import os
import datetime as dt
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoLocator
from itertools import product
import numpy as np
import csv

# Directories for reading and writing data
thedir = os.getcwd()
writedir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/interim'))
extdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/external'))
figdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'figures'))

# load data (reference data set of lcd and aws stations from 2018)
data_test = pd.read_csv(F'{extdir}/ref_2018.csv')
data_ = data_test[['Date_time_CET','BOLL_LCD_TEMP',
       'BOLL_AWS_TEMP', 'BOLL_PRECIP', 'BOLL_RADIATION_GLOBAL',
       'BOLL_WIND_SPEED_mean', 'ZOLL_STEVENSON_LCD_TEMP', 'ZOLL_2m_LCD_TEMP',
       'ZOLL_3m_LCD_TEMP', 'ZOLL_AWS_TEMP', 'ZOLL_RADIATION_GLOBAL',
       'ZOLL_SUNSHINE', 'ZOLL_PRECIP', 'ZOLL_WIND_SPEED_MEAN', 'AFU_LCD_TEMP',
       'AFU_AWS_TEMP', 'AFU_WIND_SPEED_MEAN', 'AFU_RADIATION_GLOBAL']]
data_.set_index('Date_time_CET',inplace=True)

In [2]:
class Thing:
    def __init__(self, value, index, header):
        self.name = header
        self.time = dt.datetime.strptime(index,'%d.%m.%Y %H:%M')
        self.value = value
        self.station = header[:3].lower()
        if 'LCD' in header:
            self.stype = 'lcd'
        else:
            self.stype = 'aws'
        if 'TEM' in header:
            self.qty = 'temp'
        elif 'SUN':
            self.qty = 'sun'
        elif 'RAD':
            self.qty = 'rad'
        elif 'WIN':
            self.qty = 'wind'
        elif 'PRE':
            self.qty = 'prec'
    
    def __lt__(self, other):
         return self.time < other.time
    
    #def time_date(self):
    #    return dt.datetime.strptime(self.time,'%d.%m.%Y %H:%M')
    
    def get_period(self, strfkey):
        return dt.datetime.strftime(self.time,strfkey)

In [3]:
cols = data_.columns
mylist = []
for index,rows in data_.iterrows():
        mylist.append([Thing(data_.at[index,index_c],index,index_c) for index_c in cols])

In [4]:
ndf = np.array([ x for y in mylist for x in y])

In [5]:
period_keys = {'hour':'%H','month':'%m','year':'%Y','day':'%j'}
key = period_keys['day']

temp lcd - aws differences

In [32]:

def get_diffs(station_dict):
    newdict = {}
    for key,values in station_dict.items():
        val = values[0]- values[1]
        newdict[key] = [val]
    return newdict

def get_dicts(stations,qty):
    mydicts = []
    for i in stations:
        newdict = {}
        aws = [x.value for x in ndf if (x.qty == qty) & (x.station == i) & (x.stype == 'aws')] 
        aws.sort()
        lcd = [x.value for x in ndf if (x.qty == qty) & (x.station == i) & (x.stype == 'lcd')]
        lcd.sort()
        b = [i] * len(lcd)
        values = list(zip(aws,lcd,b))
        times = list(set([x.time for x in ndf]))
        times.sort()
        mydicts.append(dict(list(zip(times,values))))
    return (mydicts)

def make_frames(stations,qty):
    dicts = get_dicts(stations,qty)
    mylist = []
    for i in dicts:
        mylist.append(get_diffs(i))
    newlist = []
    for i in mylist:
        df = pd.DataFrame.from_dict(i,orient = 'index',columns = ['-aws-lcd-diff'])
        newlist.append(df)
    df = pd.concat(newlist,axis=1)
    return df

In [33]:
test = make_frames(['bol','zol'],'temp')

In [34]:
test

,-aws-lcd-diff,-aws-lcd-diff
2018-05-16 00:00:00,-10006.9,-0.2
2018-05-16 00:10:00,-10006.9,-0.1
2018-05-16 00:20:00,-10006.9,0.1
2018-05-16 00:30:00,-10006.9,0.1
2018-05-16 00:40:00,-10007.0,0.2
...,...,...
2018-09-15 23:10:00,-0.2,18.7
2018-09-15 23:20:00,-0.1,18.7
2018-09-15 23:30:00,-0.2,18.7
2018-09-15 23:40:00,-0.1,18.8
